In [1]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
import sys
import math
sys.path.append("..") 
from scipy import stats
from utils import * 

单参数辐射模型
$$F_{ij}=O_i[\frac{m_im_j}{(m_i+S_{ij}+m_j)(m_i+S_{ij})}]^{\lambda}$$
$$O_i={\theta}Pop_i$$
两边取log
$$logF_{ij}=log{\theta}+logPop_i+\frac{m_im_j}{(m_i+S_{ij}+m_j)(m_i+S_{ij})}$$
$$logF_{ij}-logPop_i=log{\theta}+{\lambda}log\frac{m_im_j}{(m_i+S_{ij}+m_j)(m_i+S_{ij})}$$

<h3>广东省

In [4]:
region='GD'

#load city data
citydf=pd.read_csv('../Data/'+region+'/'+region+'_city_flow_detail_log.csv')
city_Dist=np.genfromtxt('../Data/'+region+'/city_adj.csv', dtype=np.float32,delimiter=',')
city_attr_df=pd.read_csv('../Data/'+region+'/'+region+'_city_attr.csv')
city_idx_map=Get_idxmap(city_attr_df)
city_flow=np.genfromtxt('../Data/'+region+'/'+'city_flow_matrix.csv', dtype=np.int32,delimiter=',')

#load county data
countydf=pd.read_csv('../Data/'+region+'/'+region+'_county_flow_detail_log.csv')
county_Dist=np.genfromtxt('../Data/'+region+'/county_adj.csv', dtype=np.float32,delimiter=',')
county_attr_df=pd.read_csv('../Data/'+region+'/'+region+'_county_attr.csv')
county_idx_map=Get_idxmap(county_attr_df)

#load relation data
county_city_rela=load_relation('../Data/'+region+'/city_county_rela.csv')

人口
$$m_i=pop_i$$

In [ ]:
variate='pop'
city_attr_list=list(city_attr_df[variate])
county_attr_list=list(county_attr_df[variate])

citydf=Fit_RMmodel(citydf,city_idx_map,city_Dist,city_attr_df,variate) 
countydf=Fit_RMmodel(countydf,county_idx_map,county_Dist,county_attr_df,variate)
citydf['y']=citydf['flow_intensity_l']-citydf['o_pop_l']
countydf['y']=countydf['flow_intensity_l']-countydf['o_pop_l']

cityres = smf.ols(formula = 'y~{}'.format(variate+'_prob_l'), data = citydf).fit()

citydf['RMflow']=np.exp(cityres.predict(citydf)+citydf['o_pop_l'])
countydf['RMflow']=np.exp(cityres.predict(countydf)+countydf['o_pop_l'])

evaluate(countydf['RMflow'],countydf['flow_intensity'])
countydf=finetuning(countydf,county_city_rela,citydf,'RMflow')
evaluate(countydf['RMflow_finetune'],countydf['flow_intensity'])

In [ ]:
variate='gdp'
city_attr_list=list(city_attr_df[variate])
county_attr_list=list(county_attr_df[variate])

citydf=Fit_RMmodel(citydf,city_idx_map,city_Dist,city_attr_df,variate) 
countydf=Fit_RMmodel(countydf,county_idx_map,county_Dist,county_attr_df,variate)
citydf['y']=citydf['flow_intensity_l']-citydf['o_pop_l']
countydf['y']=countydf['flow_intensity_l']-countydf['o_pop_l']

cityres = smf.ols(formula = 'y~{}'.format(variate+'_prob_l'), data = citydf).fit()

citydf['RMflow']=np.exp(cityres.predict(citydf)+citydf['o_pop_l'])
countydf['RMflow']=np.exp(cityres.predict(countydf)+countydf['o_pop_l'])

evaluate(countydf['RMflow'],countydf['flow_intensity'])
countydf=finetuning(countydf,county_city_rela,citydf,'RMflow')
evaluate(countydf['RMflow_finetune'],countydf['flow_intensity'])